In [1]:
import pandas as pd
import numpy as np

In [2]:
pwd

'/lab/public-heath-study/data-python'

## Question 1
Donnez le résultat de votre calcul pour l'année 2013


In [3]:
pop=pd.read_csv("/Users/yassine/Desktop/data-python/fr_population.csv")
pop=pop[['Zone','Valeur','Code zone']]
pop['Valeur']=pop['Valeur']*1000
pop=pop.rename(columns={"Valeur":"population", "Zone":"pays", "Code zone":"zone"})
pop.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/yassine/Desktop/data-python/fr_population.csv'

In [ ]:
pop['population'].sum()

In [ ]:
pop.duplicated()

In [ ]:
pop.drop_duplicates(subset=['pays'], keep= False)
pop=pop.loc[pop['pays']!= 'Chine',:]
pop.shape
pop['population'].sum()

In [ ]:
pop=pop.loc[pop['pays']!= 'Chine',:]

In [ ]:
pop.shape

In [ ]:
pop['population'].sum()

## Question 2 
Identifier les redondances

In [ ]:
# Import des données des bilans alimentaires 
veg = pd.read_csv("/Users/yassine/Desktop/data-python/fr_vegetaux.csv")
ani = pd.read_csv("/Users/yassine/Desktop/data-python/fr_animaux.csv")


# Ajout de la variable origin
ani["Origine"] = "animale"
veg["Origine"] = "vegetale"


# On regroupe veg et ani en un unique dataframe, via une union
temp = ani.append(veg)


# Suppression de ani et veg
del ani, veg


# Transformation de temp en table pivot
data = temp.pivot_table(index=["Zone","Produit","Code Produit","Origine"],
                        columns = ["Élément"], values=["Valeur"], aggfunc=sum)

data.columns = ['Aliments pour animaux', 'Autres Utilisations', 'Disponibilité alimentaire (Kcal/personne/jour)', 
                'Disponibilité alimentaire en quantité (kg/personne/an)', 
                'Disponibilité de matière grasse en quantité (g/personne/jour)', 
                'Disponibilité de protéines en quantité (g/personne/jour)', 'Disponibilité intérieure', 
                'Exportations - Quantité', 'Importations - Quantité', 'Nourriture', 'Pertes', 'Production', 
                'Semences', 'Traitement', 'Variation de stock']


dispo_alimentaire = data.reset_index()

#Nourriture = nouriture pour humain

dispo_alimentaire.head()


In [ ]:
listecolonne = ['Aliments pour animaux', 'Autres Utilisations', 'Disponibilité intérieure', 
                'Exportations - Quantité', 'Importations - Quantité', 'Nourriture', 'Pertes', 'Production', 
                'Semences', 'Traitement', 'Variation de stock']


for colonne in listecolonne :
    dispo_alimentaire[colonne] = dispo_alimentaire[colonne] * 1000000


## Question 3 
Calculez (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal puis en kg de protéines.

In [ ]:
dispo_alimentaire=dispo_alimentaire.rename(columns={"Zone":"pays"})
dispo_alimentaire=dispo_alimentaire.merge(pop,on="pays")
dispo_alimentaire.head()
dispo_alimentaire['Dispokcal']= dispo_alimentaire ['Disponibilité alimentaire (Kcal/personne/jour)']*dispo_alimentaire['population']*365
dispo_alimentaire.head()
dispo_alimentaire['Dispoperproteine']=dispo_alimentaire ['Disponibilité de protéines en quantité (g/personne/jour)']*dispo_alimentaire['population']*365/1000
dispo_alimentaire.head()

In [ ]:
dispo_alimentaire=dispo_alimentaire.merge(pop,on="pays")
dispo_alimentaire.head()

In [ ]:
dispo_alimentaire['Dispokcal'] = dispo_alimentaire ['Disponibilité alimentaire (Kcal/personne/jour)']*pop['population']*365
dispo_alimentaire.head()

In [ ]:
dispo_alimentaire['Dispoperproteine']=dispo_alimentaire ['Disponibilité de protéines en quantité (g/personne/jour)']*pop['population']*365/1000
dispo_alimentaire.head()

## Question 4 
Calculez pour chaque produit le ratio energie/poids

In [ ]:
dispo_alimentaire['Ratioenergiepoid']=dispo_alimentaire['Dispokcal']/ dispo_alimentaire ['Nourriture']
dispo_alimentaire['Ratioperproteine']=dispo_alimentaire['Dispoperproteine']/ dispo_alimentaire ['Nourriture']
dispo_alimentaire.head()
dispo_alimentaire = dispo_alimentaire.replace([np.inf, -np.inf],np.nan)
ratio =dispo_alimentaire.groupby ('Produit').mean()
ratio =ratio.reset_index()
ratio.head()
ratio =ratio[['Produit', 'Ratioenergiepoid', 'Ratioperproteine']]
ratio.loc[ratio['Produit'] == 'Oeufs',:]
ratio =ratio[['Produit', 'Ratioenergiepoid', 'Ratioperproteine']]
ratio.loc[ratio['Produit'] == 'Avoine',:]

In [ ]:
dispo_alimentaire = dispo_alimentaire.replace([np.inf, -np.inf],np.nan)
ratio =dispo_alimentaire.groupby ('Produit').mean()
ratio =ratio.reset_index()
ratio.head()

In [ ]:
ratio =ratio[['Produit', 'Ratioenergiepoid', 'Ratioperproteine']]
ratio.loc[ratio['Produit'] == 'Oeufs',:]

la teneur en proteine de l'avoine

In [ ]:
ratio =ratio[['Produit', 'Ratioenergiepoid', 'Ratioperproteine']]
ratio.loc[ratio['Produit'] == 'Avoine',:]

## Question 5
Citez 5 aliments parmi les 20 aliments les plus caloriques

In [ ]:
ratio.sort_values (by = 'Ratioenergiepoid', ascending = False)

## Question 6
Calculez pour les produits végétaux la dispo interieur mondiale exprimé en kcal

In [ ]:
dfvegetaux = dispo_alimentaire.loc[dispo_alimentaire['Origine'] == 'vegetale',:]
dfvegetaux.head()

In [ ]:
dfvegetaux['dispointerieurKcal'] = dfvegetaux ['Disponibilité intérieure'] * dfvegetaux ['Ratioenergiepoid']
dfvegetaux.head()

## Question 7
Combien d'humains pourraient être nourris si toute la disponibilité intérieur mondiale de produits végétaux était utilisé pour de la nourriture

En termes de calories sachant qu'en moyenne une personne a besoin de 2350 calories par jour

In [ ]:
resp1 = dfvegetaux['dispointerieurKcal'].sum()/365 / 2500
KG_PROT_PER_CAPITA_PER_DAY = 62 * .9 * .001
dfvegetaux['dispointerieurprot'] = dfvegetaux ['Disponibilité intérieure'] * dfvegetaux ['Ratioperproteine']
dfvegetaux.head()
resp2 = dfvegetaux['dispointerieurprot'].sum()/KG_PROT_PER_CAPITA_PER_DAY
resp1

Ce qui represente en pourcentage 

## Question 8


In [ ]:
q_kcal = dfvegetaux["dispointerieurKcal"].sum()
q_kgprot = dfvegetaux["dispointerieurprot"].sum()
quantity = pd.DataFrame({"kcal": q_kcal, "kgprot": q_kgprot}, index=[0])

quantity

In [ ]:
NB_KCAL_PER_CAPITA_PER_DAY = 2500
repQ81 = quantity["kcal"] / NB_KCAL_PER_CAPITA_PER_DAY
repQ81Pourcent = format(round(repQ81/1000000000,2))
sumkcal = quantity["kcal"].sum()
persNourrie = sumKcal / 365 
persNourrie

In [ ]:
KG_PROT_PER_CAPITA_PER_DAY = 62 * .9 * .001
repQ82 = quantity["kgprot"] / KG_PROT_PER_CAPITA_PER_DAY
repQ82Pourcent = format(round(repQ81/1000000000,2))


## question 9

In [ ]:

dispoG = dispo_alimentaire["Disponibilité de matière grasse en quantité (g/personne/jour)"].sum() + dispo_alimentaire['Disponibilité de protéines en quantité (g/personne/jour)'].sum() + (dispo_alimentaire['Disponibilité alimentaire en quantité (kg/personne/an)'].sum() * 1000) 
dispoKcal = dispo_alimentaire['Disponibilité alimentaire (Kcal/personne/jour)'].sum() * 1000000 * dispo_alimentaire["Ratioenergiepoid"]

quantityQ9 = pd.DataFrame({"kcal": dispoKcal, "kgprot": dispoG}, index=[0])


quantityQ9['kcal']


In [ ]:
NB_KCAL_PER_CAPITA_PER_DAY = 2500
repQ91 = quantityQ9["kcal"] / NB_KCAL_PER_CAPITA_PER_DAY
repQ91Pourcent = format(round(repQ91/1000000000,2))

KG_PROT_PER_CAPITA_PER_DAY = 62 * .9 * .001
repQ92 = quantityQ9["kgprot"] / KG_PROT_PER_CAPITA_PER_DAY
repQ92Pourcent = format(round(repQ92/1000000000,2))

repQ91Pourcent

## Question 10

In [ ]:
sousAlimentationCsv = pd.read_csv("/Users/yassine/Desktop/data-python/fr_sousalimentation.csv")
sousAlimentationCsv

In [ ]:
sousAlimentationCsv = pd.read_csv("/Users/yassine/Desktop/data-python/fr_sousalimentation.csv")
sousAlimentationCsv=sousAlimentationCsv[['Code année','Code zone','Zone','Produit','Valeur','Année']]
sousAlimentationCsv["Valeur"] = pd.to_numeric(sousAlimentationCsv["Valeur"],errors='coerce')
sousAlimentationCsv['underfeed'] = sousAlimentationCsv['Valeur'] * 1000000
sousAlimentationCsv = sousAlimentationCsv.loc[(sousAlimentationCsv['Code année'] == 20122014 )]
sousAlimentationCsv = sousAlimentationCsv.groupby(['Zone']).sum()
sousAlimentationCsv

In [ ]:
sousAlimentationCsv = sousAlimentationCsv.loc[(sousAlimentationCsv['Code année'] == 20122014 )]
sousAlimentationCsv = sousAlimentationCsv.groupby(['Zone']).sum()
sousAlimentationCsv

In [ ]:
pop=pd.read_csv("/Users/yassine/Desktop/data-python/fr_population.csv")
pop=pop[['Zone','Valeur','Code zone']]
pop['Valeur']=pop['Valeur']*1000
pop=pop.rename(columns={"Valeur":"population", "Zone":"pays", "Code zone":"zone"})
pop

In [ ]:

##
sousAlimentationPopulation = pd.merge(left=sousAlimentationCsv, right=pop, left_on='Code zone', right_on='zone')
sousAlimentationPopulation['ProportionSsAlimentPopulation'] = (sousAlimentationPopulation['underfeed/perYear'] * 100) / sousAlimentationPopulation['population']

## moyenne Pourcentage Mondial des personne sur une année

pourcentSousAlimentationWorld = (sousAlimentationPopulation['underfeed/perYear'].sum() * 100) / sousAlimentationPopulation['population'].sum()

sousAlimentationPopulation


## Question 11

In [ ]:


## http://www.fao.org/in-action/inpho/crop-compendium/cereals-grains/fr/?page=1&ipp=7&no_cache=1&tx_dynalist_pi1[par]=YToxOntzOjE6IkwiO3M6MToiMSI7fQ==

##Repérez dans vos données les informations concernant les céréales (par exemple en créant une colonne de type booléen nommée "is_cereal").

## : En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), 
cerealsCsv = pd.read_csv("/Users/yassine/Desktop/data-python/fr_céréales.csv")

cerealsCsv['is_cereals'] = 'false'
cerealsCsv.loc[cerealsCsv['Produit'] == 'Maïs', ['is_cereals']] = 'true'
cerealsCsv.loc[cerealsCsv['Produit'] == 'Millet', ['is_cereals']] = 'true'
cerealsCsv.loc[cerealsCsv['Produit'] == 'Blé', ['is_cereals']] = 'true'
cerealsCsv.loc[cerealsCsv['Produit'] == 'Riz (Eq Blanchi)', ['is_cereals']] = 'true'
cerealsCsv.loc[cerealsCsv['Produit'] == 'Orge', ['is_cereals']] = 'true'
cerealsCsv.loc[cerealsCsv['Produit'] == 'Sorgho', ['is_cereals']] = 'true'
cerealsCsv = cerealsCsv.rename(columns={"Valeur":"populationSsAlimenter"})

cerealsCsv

In [ ]:
cerealsCsv = cerealsCsv[['Code zone','Zone','Unité','populationSsAlimenter','Code Produit','is_cereals']]
cerealsCsv

In [ ]:
daAnimal = dispo_alimentaire.loc[dispo_alimentaire['Origine'] == 'animale',:]
daAnimal['Aliments pour animaux'] = daAnimal['Aliments pour animaux'].astype(float)

In [ ]:
animalCereal = pd.concat([daAnimal, cerealsCsv], axis=1, join='inner')
animalCereal

In [ ]:
##En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), 
##quelle proportion (en termes de poids) est destinée à l'alimentation animale 

isCereal = animalCereal.loc[animalCereal['is_cereals'] == 'true',:]
tchekValuesAnimalCereal = isCereal.loc[isCereal['Aliments pour animaux'] > 0,:]
sumAlimentPourAnimaux = tchekValuesAnimalCereal['Aliments pour animaux'].sum()
tchekValuesAnimalCereal['ProportionFoodAnimalsByGlobalFood'] = tchekValuesAnimalCereal['Nourriture'] /  (tchekValuesAnimalCereal['Nourriture'] + tchekValuesAnimalCereal['Aliments pour animaux'])
tchekValuesAnimalCereal


In [ ]:
sumAlimentPourAnimaux

In [ ]:
sumNourritues = tchekValuesAnimalCereal['Nourriture'].sum()
globalproportionFoodAnimalsByGlobalFood = sumNourritues /  (sumAlimentPourAnimaux + sumAlimentPourAnimaux)

globalproportionFoodAnimalsByGlobalFood

In [ ]:
## Sélectionnez parmi les données des bilans alimentaires les informations .
## relatives aux pays dans lesquels la FAO recense des personnes en sous-nutrition


dispoSsAlimPop = pd.concat([dispo_alimentaire, sousAlimentationPopulation], axis=1, join='inner')

dispoSsAlimPop.columns


In [ ]:
unsetImportationNoneValues = dispoSsAlimPop.loc[dispoSsAlimPop['Importations - Quantité'] > 0,:]
unsetImportationNoneValues.sort_values('Importations - Quantité')
unsetImportationNoneValues
importations = unsetImportationNoneValues[['Code zone','Zone','Produit','Importations - Quantité']]
rankImportations = importations.sort_values(by=['Importations - Quantité'], ascending=False)[:15]
rankImportations



In [ ]:
##Parmi les données des bilans alimentaires au niveau mondial, sélectionnez 
##les 200 plus grandes importations de ces produits 
##(1 importation = une quantité d'un produit donné importée par un pays donné)

rankImportation200 = importations.sort_values(by=['Importations - Quantité'])[:200]

rankImportation200


##rep 

##imp = data[(data.item.isin(exported_products)) & (data.year == YEAR)]
##imp.sort_values("Import Quantity").iloc[200:,]

In [ ]:

pd = rankImportation200.groupby("Produit")

pd

In [ ]:
dispo_alimentaire.columns
dispo_alimentaire=dispo_alimentaire.rename(columns={"Zone":"zone"})

In [ ]:
sousAlimentationPopulation

## fin de la 11

## Question 12

In [ ]:
ratiodisp_inter = dispo_alimentaire[['produit', 'ratioDispoInterieur']].sort_values('ratioDispoInterieur', ascending=False)[:3]

ratioanim_hum = dispo_alimentaire[['produit', 'ratioAlimentPourAnimaux']].sort_values('ratioAlimentPourAnimaux', ascending=False)\[:3]

## Question 13

In [ ]:

qt_cere_pr_anim_ktonne = dispo_alimentaire.loc[
    (dispo_alimentaire.pays == "États-Unis d'Amérique")
    & (dispo_alimentaire.is_cereals == true),
    "alim_pr_anim__ktonne"
].sum()

qt_cere_pr_anim_ktonne

## question 14

In [ ]:
thailande = dispo_alimentaire.loc[
    (dispo_alimentaire'pays'] == 'Thaïlande')
    &(dispo_alimentaire['produit'] == 'Manioc'),
].iloc[0]

proportion_manioc_export = thailande['export__ktonne'] / thailande['prod__ktonne']
proportion_per_ss_alim = thailande['pop_ss_alim'] / sr['pop']